<a href="https://colab.research.google.com/github/Vkshah20/Intel-Image-Classification/blob/master/Intel_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import  Callback, TensorBoard


In [ ]:
class MyCall(Callback):
    def on_epoch_end(self,model, logs={}):
        if(logs['acc'] > 0.95):
            self.model.stop_training = True

calls = MyCall()
tbCall = TensorBoard('log', histogram_freq = 1)


In [ ]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('drive/My Drive/DATA.zip', 'r') as zipObj:
    zipObj.extractall()

In [ ]:
trainP = 'DATA/seg_train'
testP = 'DATA/seg_test'

trainDataGen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    zoom_range=0.2,)
testDataGen = ImageDataGenerator(rescale = 1./255)

trainGen = trainDataGen.flow_from_directory(
    trainP,
    target_size = (150,150),
    batch_size = 64,
    class_mode = 'categorical')

testGen = testDataGen.flow_from_directory(
    testP,
    target_size = (150,150),
    batch_size = 64,
    class_mode = 'categorical')



Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), input_shape = (150,150, 3), activation = 'relu'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation = 'relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation = 'relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation = 'relu'),
    Dropout(0.5),
    Dense(128, activation = 'relu'),
    Dense(6, activation = 'softmax'),
    ])
model.compile('adam', 'categorical_crossentropy', ['acc'])


In [ ]:
from google.colab import files

!zip -r log.zip log


  adding: log/ (stored 0%)
  adding: log/train/ (stored 0%)
  adding: log/train/events.out.tfevents.1592206628.bc0914672ea0.119.269.v2 (deflated 84%)
  adding: log/train/plugins/ (stored 0%)
  adding: log/train/plugins/profile/ (stored 0%)
  adding: log/train/plugins/profile/2020_06_15_07_37_15/ (stored 0%)
  adding: log/train/plugins/profile/2020_06_15_07_37_15/bc0914672ea0.trace.json.gz (deflated 0%)
  adding: log/train/plugins/profile/2020_06_15_07_37_15/bc0914672ea0.tensorflow_stats.pb (deflated 72%)
  adding: log/train/plugins/profile/2020_06_15_07_37_15/bc0914672ea0.overview_page.pb (deflated 59%)
  adding: log/train/plugins/profile/2020_06_15_07_37_15/bc0914672ea0.kernel_stats.pb (deflated 92%)
  adding: log/train/plugins/profile/2020_06_15_07_37_15/bc0914672ea0.input_pipeline.pb (deflated 58%)
  adding: log/train/events.out.tfevents.1592206635.bc0914672ea0.profile-empty (deflated 5%)
  adding: log/validation/ (stored 0%)
  adding: log/validation/events.out.tfevents.1592206734.b

In [ ]:
model.fit_generator(
    trainGen,
    epochs = 1000,
    callbacks = [calls, tbCall],
    validation_data = testGen,)
model.save('kaggle.hdf5')


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/1000
220/220 [==============================] - 100s 452ms/step - loss: 1.1676 - acc: 0.5316 - val_loss: 0.9000 - val_acc: 0.6320
Epoch 2/1000
220/220 [==============================] - 99s 449ms/step - loss: 0.9052 - acc: 0.6611 - val_loss: 0.7704 - val_acc: 0.7193
Epoch 3/1000
220/220 [==============================] - 98s 448ms/step - loss: 0.7780 - acc: 0.7144 - val_loss: 0.7530 - val_acc: 0.7293
Epoch 4/1000
220/220 [==============================] - 98s 444ms/step - loss: 0.7242 - acc: 0.7349 - val_loss: 0.6155 - val_acc: 0.7830
Epoch 5/1000
220/220 [==============================] - 97s 441ms/step - loss: 0.6632 - acc: 0.7545 - val_loss: 0.6668 - val_acc: 0.7677
Epoch 6/1000
220/220 [==============================] - 97s 442ms/step - loss: 0.6166 - acc: 0.7772 - val_loss: 0.5003 - val_acc: 0.8203
Epoch 7/1000
220/220 [==============================] - 97s 439ms/step - loss: 0.5802 - acc: 0.7907 -

In [ ]:
!zip -r logs.zip logs

  adding: logs/ (stored 0%)
  adding: logs/train/ (stored 0%)
  adding: logs/train/events.out.tfevents.1600325235.0cb1e2e845c1.1985.314.v2 (deflated 84%)
  adding: logs/train/events.out.tfevents.1600325892.0cb1e2e845c1.1985.14107.v2 (deflated 90%)
  adding: logs/train/events.out.tfevents.1600325240.0cb1e2e845c1.profile-empty (deflated 8%)
  adding: logs/train/plugins/ (stored 0%)
  adding: logs/train/plugins/profile/ (stored 0%)
  adding: logs/train/plugins/profile/2020_09_17_06_59_19/ (stored 0%)
  adding: logs/train/plugins/profile/2020_09_17_06_59_19/0cb1e2e845c1.overview_page.pb (deflated 60%)
  adding: logs/train/plugins/profile/2020_09_17_06_59_19/0cb1e2e845c1.xplane.pb (deflated 81%)
  adding: logs/train/plugins/profile/2020_09_17_06_59_19/0cb1e2e845c1.input_pipeline.pb (deflated 57%)
  adding: logs/train/plugins/profile/2020_09_17_06_59_19/0cb1e2e845c1.memory_profile.json.gz (stored 0%)
  adding: logs/train/plugins/profile/2020_09_17_06_59_19/0cb1e2e845c1.trace.json.gz (deflate

In [ ]:
!cp -r kaggle.hdf5 '/content/drive/My Drive/'